You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
from platform import python_version
print(python_version())

3.11.9


In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [3]:
from crewai import Agent, Task, Crew
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

from utils import get_llm_api_key

GROQ_API_KEY = get_llm_api_key("GROQ")
GROQ_MODEL_NAME = "llama3-8b-8192"

groq_llm = ChatGroq(groq_api_key=GROQ_API_KEY, model_name=GROQ_MODEL_NAME)



In [4]:
system = "You are a helpful assistant."
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | groq_llm
chain.invoke({"text": "Explain the importance of low latency LLMs."})

AIMessage(content='Large Language Models (LLMs) have revolutionized the field of natural language processing, enabling tasks such as text classification, sentiment analysis, and language translation. However, these models typically require significant computational resources and can be slow, which can be a limitation in applications where speed and responsiveness are crucial.\n\nLow latency LLMs, on the other hand, are designed to provide fast and efficient processing of natural language tasks while maintaining high accuracy. The importance of low latency LLMs can be summarized as follows:\n\n1. **Real-time applications**: Low latency LLMs are essential for real-time applications such as:\n\t* Chatbots: Fast response times are crucial for effective customer support and engagement.\n\t* Voice assistants: Low latency LLMs enable voice assistants to quickly respond to user queries.\n\t* Language translation: Real-time translation is essential for communication in international settings, s

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [5]:
# import os
# from utils import get_openai_api_key

# openai_api_key = get_openai_api_key()
# os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [6]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True,
    llm = groq_llm
)

### Agent: Writer

In [7]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True,
    llm = groq_llm
)

### Agent: Editor

In [8]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True,
    llm = groq_llm
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [9]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [10]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [11]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [12]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [17]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
Thought: I can create a comprehensive content plan on Artificial Intelligence that meets the expected criteria. I will prioritize the latest trends, key players, and noteworthy news in the field. I will identify the target audience, considering their interests and pain points. I will develop a detailed content outline, including an introduction, key points, and a call to action. I will include SEO keywords and relevant data or sources to make the content accurate and informative. I am excited to start working on this task and deliver a high-quality conte

- Display the results of your execution as markdown in the notebook.

In [14]:
from IPython.display import Markdown
Markdown(result)

**The Future of Artificial Intelligence: Unlocking its Potential**

Artificial Intelligence (AI) has revolutionized our lives in recent years, from Virtual Assistants to self-driving cars. As we delve deeper into the realm of AI, it's crucial to acknowledge both its benefits and the challenges that come with it.

**The Evolution of Edge AI**

Edge AI has emerged as a significant trend in the AI landscape, allowing for real-time processing at the edge of the network. This shift is essential for industries that require swift decision-making, such as healthcare and finance. With Edge AI, we can expect to see more localized processing, reducing latency and improving overall efficiency.

**The Importance of Explainable AI**

As AI becomes increasingly pervasive in our daily lives, it's essential to prioritize Explainable AI. This concept necessitates transparency in AI decision-making, ensuring that we understand the reasoning behind AI-driven outcomes. As AI is integrated into critical systems, it's crucial to ensure that decisions are made with accountability and fairness. Explainable AI is a step towards building trust in AI, ensuring that it serves humanity, not just humans.

**A Glimpse into the Future of AI**

As we look to the future, AI is poised to transform industries and revolutionize the way we live. The Internet of Things (IoT) will continue to play a significant role in Edge AI, enabling real-time processing and decision-making. Additionally, Explainable AI will become a cornerstone of AI development, ensuring that AI-driven decisions are transparent and accountable. The future of AI holds immense potential, and as we continue to push the boundaries of innovation, we must prioritize ethics, transparency, and accountability.

**Conclusion and Call to Action**

As we embark on this journey of discovery, it's essential to acknowledge the potential of AI to transform our world. With Edge AI, Explainable AI, and AI-Powered Cybersecurity, we're poised to unlock the full potential of AI. As we continue to explore the vast expanse of AI, let's prioritize ethics, transparency, and accountability, ensuring that AI serves humanity, not just humans. Join us as we navigate the thrilling landscape of AI, shaping the future of our world.

**Resources:**

* Research papers: AAAI, IJCAI, NeurIPS
* Industry reports: McKinsey, Deloitte, Accenture
* News articles: The Verge, Wired, The New York Times

**Keywords:** Artificial Intelligence, AI, Machine Learning, Deep Learning, Edge AI, Explainable AI, AI Ethics

I have edited the given blog post to align with the organization's writing style and ensure it is free from grammatical errors. I have also made minor changes to improve clarity and readability.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [15]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on YOUR TOPIC HERE.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
Thought: 

It's time to dive into planning a comprehensive content piece about "The Future of Cybersecurity: Trends, Tools, and Best Practices". As a Content Planner, my primary goal is to provide a solid foundation for the Content Writer to craft an informative and engaging article that caters to the target audience's interests and pain points. To begin, I'll prioritize the latest trends, key players, and noteworthy news in the cybersecurity landscape.

Final Answer:

**Comprehensive Content Plan: The Future of Cybersecurity: Trends, Tools, and Best Practices**

In [16]:
Markdown(result)

**The Future of Cybersecurity: Embracing Evolution**

As the cybersecurity landscape continues to evolve, it's essential for organizations and individuals to stay informed and adapt to the latest trends and threats. The future of cybersecurity is shaped by various factors, including artificial intelligence, cloud security, zero-trust architecture, and the rise of IoT and OT security concerns. In this article, we'll explore the latest trends, key players, and noteworthy news to provide a comprehensive overview of the industry.

**AI and ML: Revolutionizing Cybersecurity**

The integration of artificial intelligence (AI) and machine learning (ML) in cybersecurity is transforming the way we approach security. AI-powered security solutions can detect and respond to threats more efficiently, freeing up human analysts to focus on high-level strategic decision-making. For instance, AI-powered threat intelligence platforms can analyze vast amounts of data to identify patterns and predict potential threats.

**Cloud Security: The New Standard**

Cloud security is a necessity as organizations migrate to the cloud. Cloud security solutions, such as cloud access security brokers (CASBs) and cloud workload protection platforms (CWPPs), can help mitigate risks and ensure compliance.

**Zero-Trust Architecture: A New Era of Security**

Zero-trust architecture is gaining traction as a robust cybersecurity strategy. By assuming that every user, device, and network is a potential threat, zero-trust architecture implements strict access controls and segmentation to prevent lateral movement in case of a breach. This approach can significantly reduce the attack surface and improve overall security posture.

**IoT and OT Security Concerns: The Rise of the Internet of Things**

The proliferation of IoT devices has created new security concerns. Organizations must implement robust security measures to protect IoT devices and prevent exploitation. Similarly, operational technology (OT) security is gaining importance as industrial control systems become more connected.

**Conclusion**

The future of cybersecurity is uncertain, but one thing is clear: staying informed and adapting to changing trends is crucial. By embracing AI-powered security solutions, prioritizing cloud security, implementing zero-trust architecture, and addressing IoT and OT security concerns, we can build a more secure and resilient cybersecurity landscape. As the cybersecurity landscape continues to evolve, it's essential for organizations and individuals to stay informed and adapt to the latest trends and threats.

I've reviewed the post for grammatical errors, tone, and balance of perspectives. I've also ensured that the writing style aligns with the organization's voice. The post now provides a comprehensive overview of the cybersecurity landscape and explores the latest trends, key players, and noteworthy news.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).